### Resources
https://pymotw.com/3/string/index.html

https://docs.python.org/3/library/string.html

https://qz.com/884448/every-successful-relationship-is-successful-for-the-same-exact-reasons/

http://www.regular-expressions.info/refext.html

magic methods: https://docs.python.org/3/reference/datamodel.html#object.__getitem__

oop: http://www.itmaybeahack.com/book/python-2.6/html/index.html


documentation: https://medium.com/python-pandemonium/python-introspection-with-the-inspect-module-2c85d5aa5a48#.umcbxc6uj


unicode, bytes, strings

http://stackoverflow.com/questions/8219706/difference-between-binary-string-byte-string-unicode-string-and-an-ordinary-st

It depends on the version on Python you are using.
In Python 2.x if you write 'abc' it has type str but this means a byte string. If you want a Unicode string you must write u'abc'.
In Python 3.x if you write 'abc' it still has type str but now this means that is a string of Unicode characters. If you want a byte string you must write b'abc'. It is not allowed to write u'abc'.

In [54]:
import string

s = """what is shining on the moon?      Abolo!!!"""
print(s)
print()
print(string.capwords(s))

what is shining on the moon?      Abolo!!! $ikeke

What Is Shining On The Moon? Abolo!!! $ikeke


### Example 2: template

In [63]:
import string

values = {'name' : 'OLODO RAPATA',
         'grade' : 0,
         'discipline' : "WHO KNOWS?"}

p = string.Template("""
This boy, $name, has really done well for himself. He got a $grade in $discipline
The escape character is $$.
""")

print('Assessment: ', p.substitute(values))

Assessment:  
This boy, OLODO RAPATA, has really done well for himself. He got a 0 in WHO KNOWS?
The escape character is $.



### Example 3: Safe substitution

In [58]:
import string

values = {'var': 'foo'}

t = string.Template("$var is here but $missing is not provided")

try:
    print('substitute()     :', t.substitute(values))
except KeyError as err:
    print('ERROR:', str(err))
print('safe_substitute():', t.safe_substitute(values))
print('----' * 10)

# print('Now to check for patterns')

# print('The template pattern is: \n', string.Template.pattern)
# print('----' * 10)
# print('The template pattern type is: \n', type(string.Template.pattern))
# print()
# print('----' * 10)
# print('We go a step further to reveal the makeup of the pattern')
# print()
# print('The string template string is: \n', string.Template.pattern.pattern)
# print('----' * 10)
# print('The string template string type is: \n', type(string.Template.pattern.pattern))

ERROR: 'missing'
safe_substitute(): foo is here but $missing is not provided
----------------------------------------


### What the regexes stand for

**(?P&lt;name&gt;regex)** === Captures the text matched by "regex" into the group "name". The name can contain letters and numbers but must start with a letter.

**(?:abc)** === non-capturing group


\\\$(?: === this dollar sign precedes all capturing groups, though the larger group is non-capturing due to the ?:

    (?P<escaped>\$) === overall regex \$$

    (?P<named>[_a-z][_a-z0-9]*) === the overall regex is \$(?P<named>[_a-z][_a-z0-9]*), (for safe substitution)???

    {(?P<braced>[_a-z][_a-z0-9]*)} === overall regex is \${(?P<braced>[_a-z][_a-z0-9]*)}, replaces brace content

    (?P<invalid>)
   )

In [61]:
### Borrowed
from string import Template
import re

class TemplateClone(Template):
    delimiter = '$'
    pattern = r'''
    \$(?:
      (?P<escaped>\$) |   # Escape sequence of two delimiters
      (?P<named>[_a-z][_a-z0-9]*)      |   # delimiter and a Python identifier
      {(?P<braced>[_a-z][_a-z0-9]*)}   |   # delimiter and a braced identifier
      (?P<invalid>)              # Other ill-formed delimiter exprs
    )
    '''

class TemplateAlternative(Template):
    delimiter = '[-'
    pattern = r'''
    \[-(?:
       (?P<escaped>-) |            # Expression [-- will become [-
       (?P<named>[^\[\]\n-]+)-\] | # -, [, ], and \n can't be used in names
       \b\B(?P<braced>) |          # Braced names disabled
       (?P<invalid>)               #
    )
    '''

t = TemplateClone("$hi sir")
print(t.substitute({"hi": "hello"}))
print()
ta = TemplateAlternative("[-hi-] sir")
print(ta.substitute({"hi": "have a nice day"}))
print()
ta = TemplateAlternative("[--[-hi-]-]")
print(ta.substitute({"hi": "have a nice day"}))

hello sir

have a nice day sir

[-have a nice day-]
